In [1]:
!pip install transformers torch datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split
from tensorflow.data import Dataset
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import shutil

E0000 00:00:1721646060.029222      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0722 11:01:00.037474375      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0722 11:01:00.037490957      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0722 11:01:00.037494298      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0722 11:01:00.037496792      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0722 11:01:00.037499166      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [3]:
# Ініціалізація TPU
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1721646096.875629      13 service.cc:145] XLA service 0x5abce6ab7ab0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721646096.875685      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1721646096.875691      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1721646096.875694      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1721646096.875697      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1721646096.875700      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1721646096.875702      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1721646096.875705      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1721646096.875708      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [4]:
# Load the dataset
file_path = '/kaggle/input/big-data/combined_train_big.csv'  # Update the path to your dataset
data = pd.read_csv(file_path)

# Preprocess the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment_text'].tolist(), 
    data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist(), 
    test_size=0.1
)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize function
def tokenize_texts(texts, labels, tokenizer, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    return (
        np.array(input_ids),
        np.array(attention_masks),
        np.array(labels)
    )

# Tokenize the datasets
train_input_ids, train_attention_masks, train_labels = tokenize_texts(train_texts, train_labels, tokenizer)
val_input_ids, val_attention_masks, val_labels = tokenize_texts(val_texts, val_labels, tokenizer)



# Create TensorFlow datasets
def create_tf_dataset(input_ids, attention_masks, labels, batch_size=16):
    dataset = Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_masks}, labels))
    dataset = dataset.shuffle(buffer_size=len(input_ids)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

BATCH_SIZE = 32

train_dataset = create_tf_dataset(train_input_ids, train_attention_masks, train_labels, BATCH_SIZE)
val_dataset = create_tf_dataset(val_input_ids, val_attention_masks, val_labels, BATCH_SIZE)


In [5]:
"""#  Мітки
y_train = data.iloc[:, 2:]

# Ініціалізація словника для ваг класів
class_weights_dict = {}

# Обчислення ваг для кожного класу
for i, column in enumerate(y_train.columns):
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[column]), y=y_train[column])
    class_weights_dict[i] = class_weights[1]  # class_weight повертає ваги для обох класів 0 і 1, нас цікавить вага для класу 1"""

"#  Мітки\ny_train = data.iloc[:, 2:]\n\n# Ініціалізація словника для ваг класів\nclass_weights_dict = {}\n\n# Обчислення ваг для кожного класу\nfor i, column in enumerate(y_train.columns):\n    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train[column]), y=y_train[column])\n    class_weights_dict[i] = class_weights[1]  # class_weight повертає ваги для обох класів 0 і 1, нас цікавить вага для класу 1"

In [6]:
EPOCHS = 5
with strategy.scope():
    # Load pre-trained model
    model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
    
    # Change the classifier layer to include sigmoid activation
    model.classifier.activation = tf.keras.activations.sigmoid

    # Optimizer and loss
    steps_per_epoch = len(train_texts) // BATCH_SIZE
    num_train_steps = steps_per_epoch * EPOCHS
    num_warmup_steps = int(num_train_steps * 0.1)
    optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps = num_warmup_steps, num_train_steps=num_train_steps)
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    # Compile model
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Suppress AutoGraph warnings
@tf.autograph.experimental.do_not_convert
def train_model():
    with strategy.scope():
        # Training
        history = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

        # Evaluate the model
        val_loss, val_acc = model.evaluate(val_dataset)
        print(f'Validation loss {val_loss} accuracy {val_acc}')
    
        # Save the model
    model_save_path = '/kaggle/working/saved_model'
    model.save(model_save_path)

    # Archive the model
    archive_path = '/kaggle/working/saved_model_archive.zip'
    shutil.make_archive('saved_model_archive', 'zip', model_save_path)
    print(f'Model saved and archived at {archive_path}')

train_model()


I0000 00:00:1721646465.109426      13 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-07-22 11:09:10.020954: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.
I0000 00:00:1721646553.205503     818 tpu_compilation_cache_interface.cc:441] TPU host compilation cache miss: cache_key(1ed4b6467e6a477:0:0), session_name()
I0000 00:00:1721646587.619427     818 tpu_compile_op_common.cc:245] Compilation of 1ed4b6467e6a477:0:0 with session name  took 34.413876971s and succeeded
I0000 00:00:1721646587.709932     818 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(1ed4b6467e6a477:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_train_function_4995373692512786891", property.function_library_fingerprint = 9965554627441909027, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topo

...

6288/6288 [==============================] - 974s 138ms/step - loss: 0.0879 - accuracy: 0.9071 - val_loss: 0.0426 - val_accuracy: 0.9955


I0000 00:00:1721647452.420841     780 tpu_compile_op_common.cc:245] Compilation of a4b6547fa621db51:0:0 with session name  took 3.872825612s and succeeded
I0000 00:00:1721647452.450142     780 tpu_compilation_cache_interface.cc:475] TPU host compilation cache: compilation complete for cache_key(a4b6547fa621db51:0:0), session_name(), subgraph_key(std::string(property.function_name) = "cluster_test_function_2009172832449519177", property.function_library_fingerprint = 17168402920806792758, property.mlir_module_fingerprint = 0, property.num_replicas = 8, topology.chip_bounds().x = 2, topology.chip_bounds().y = 2, topology.chip_bounds().z = 1, topology.wrap().x = false, topology.wrap().y = false, topology.wrap().z = false, std::string(property.shapes_prefix) = "3,512,;3,512,;3,6,;", property.guaranteed_constants_size = 0, embedding_partitions_fingerprint = "1688352644216761960")
I0000 00:00:1721647452.450189     780 tpu_compilation_cache_interface.cc:541] After adding entry for key a4b6547

Epoch 2/5
6288/6288 [==============================] - 810s 129ms/step - loss: 0.0373 - accuracy: 0.9912 - val_loss: 0.0400 - val_accuracy: 0.9935
Epoch 3/5
6288/6288 [==============================] - 812s 129ms/step - loss: 0.0287 - accuracy: 0.9911 - val_loss: 0.0417 - val_accuracy: 0.9931
Epoch 4/5
6288/6288 [==============================] - 812s 129ms/step - loss: 0.0212 - accuracy: 0.9885 - val_loss: 0.0486 - val_accuracy: 0.9816
Epoch 5/5
699/699 [==============================] - 25s 35ms/step - loss: 0.0550 - accuracy: 0.9775
Validation loss 0.055008940398693085 accuracy 0.9774547219276428
INFO:tensorflow:Assets written to: /kaggle/working/saved_model/assets


INFO:tensorflow:Assets written to: /kaggle/working/saved_model/assets


Model saved and archived at /kaggle/working/saved_model_archive.zip


In [13]:
# Function to predict on a new comment
def predict_comment(comment, model, tokenizer):
    encoding = tokenizer.encode_plus(
        comment,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='tf',
        truncation=True
    )
    
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    
    preds = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
    
    print(f'Predictions shape: {preds.shape}')  # Print the shape of predictions
    
    if preds.shape[0] == 1:
        preds = preds[0]  # Get the first element if it's a batch

    return {
        'toxic': preds[0],
        'severe_toxic': preds[1],
        'obscene': preds[2],
        'threat': preds[3],
        'insult': preds[4],
        'identity_hate': preds[5]
    }

# Example usage
comment = "Your example comment here"
prediction = predict_comment(comment, model, tokenizer)
print(prediction)


1/1 [==============================] - 0s 461ms/step
Predictions shape: (1, 6)
{'toxic': 0.0002438724, 'severe_toxic': 6.368756e-05, 'obscene': 8.225441e-05, 'threat': 2.8580427e-05, 'insult': 9.840727e-05, 'identity_hate': 6.3210726e-05}


In [16]:
# Example usage
comment = "many think that he is a piece of shit and behaves like a bitch. but he's actually a nice guy"
prediction = predict_comment(comment, model, tokenizer)
print(prediction)

1/1 [==============================] - 0s 479ms/step
Predictions shape: (1, 6)
{'toxic': 0.9657171, 'severe_toxic': 0.009735912, 'obscene': 0.9709351, 'threat': 0.00028795004, 'insult': 0.8563639, 'identity_hate': 0.00093352795}


In [19]:
# Generate predictions for the validation set and compute classification report
def evaluate_model(model, val_dataset, val_labels):
    predictions = []
    true_labels = []
    
    for batch in val_dataset:
        input_ids = batch[0]['input_ids']
        attention_mask = batch[0]['attention_mask']
        labels = batch[1]
        
        preds = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
        
        predictions.extend(preds)
        true_labels.extend(labels.numpy())
    
    predictions = np.array(predictions)
    true_labels = np.array(true_labels)
    
    # Binarize predictions
    predictions = np.where(predictions > 0.5, 1, 0)
    
    # Print classification report
    report = classification_report(true_labels, predictions, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], zero_division=0)
    print(report)

evaluate_model(model, val_dataset, val_labels)

1/1 [==============================] - 0s 486ms/step
               precision    recall  f1-score   support

        toxic       0.78      0.82      0.80      2175
 severe_toxic       0.49      0.37      0.42       194
      obscene       0.80      0.82      0.81      1231
       threat       0.61      0.64      0.62        67
       insult       0.76      0.76      0.76      1136
identity_hate       0.61      0.54      0.57       205

    micro avg       0.76      0.77      0.77      5008
    macro avg       0.67      0.66      0.66      5008
 weighted avg       0.76      0.77      0.77      5008
  samples avg       0.07      0.07      0.07      5008



In [20]:
from IPython.display import FileLink

file_path = '/kaggle/working/saved_model_archive.zip'
display(FileLink(file_path))


/kaggle/working/saved_model_archive.zip

In [21]:
import shutil
from IPython.display import FileLink

# Define the path where the model will be saved
model_save_path = '/kaggle/working/saved_model2'

# Save the trained model
model.save(model_save_path)

# Archive the model
archive_path = '/kaggle/working/saved_model_archive2.zip'
shutil.make_archive('/kaggle/working/saved_model_archive2', 'zip', model_save_path)
print(f'Model saved and archived at {archive_path}')

# Create a downloadable link
display(FileLink(archive_path))


INFO:tensorflow:Assets written to: /kaggle/working/saved_model2/assets


INFO:tensorflow:Assets written to: /kaggle/working/saved_model2/assets


Model saved and archived at /kaggle/working/saved_model_archive2.zip


/kaggle/working/saved_model_archive2.zip